# 複数の時系列CSVデータを同じ時間でまとめる

## 方法 (日単位データの場合)
1. 読み込むデータを全て読み込む
2. 各データの最初の日付・最後の日付を比較
3. 各データの中で最も遅い最初の日付$t_0$・最も早い最後の日付$t_f$を読み込み範囲とする
4. 各データについて，最初の日付と$t_0$の日数$\Delta{t_0}$，最後の日付と$t_f$の日数$\Delta{t_f}$を求める
5. 各データの$\Delta{t_0}$番目から$t_0$に対応するデータを探索
    1. 探索したデータの日付が$t_0$よりも早/遅ければ，日数差$\Delta{t}$だけ遅/早いデータを次に調べる
    2. 該当日のデータがなかったとき，諦める
6. 該当日が見つかったら
    1. 1日ずつ増やしてデータの日付が対応しているのかのチェック
        + 該当日なら，配列に保存
        + 該当日でなかったら，5.を行う．

# 使用モジュール
+ pandas
+ numpy
+ datetime

## CSVの読み込み

まずは，CSVの読み込みに必要なモジュールをインポートします．

In [3]:
import pandas as pd
import numpy as np

今回は，[株式投資メモ](https://kabuoji3.com/data.php?code=7203)様からダウンロードしました．
拾ってきたデータは年単位の日足データとなっていたので，まずは整形して5年分のデータとしました．

pandasモジュールを用いて読み込みます．

In [4]:
csv7203_2014 = pd.read_csv(
    "C:/Users/issei/Projects/BitTrader/Stock_Data/7203_2014.csv",
    encoding='SHIFT_JIS',
    header=1
)
print(csv7203_2014)


             日付    始値    高値    安値    終値       出来高  終値調整値
0    2014-01-06  6360  6400  6280  6300  12249300   6300
1    2014-01-07  6270  6340  6260  6270   7891400   6270
2    2014-01-08  6310  6320  6260  6300   7184100   6300
3    2014-01-09  6310  6340  6260  6270   8653000   6270
4    2014-01-10  6260  6310  6250  6290   7815900   6290
5    2014-01-14  6200  6201  6114  6144  16507500   6144
6    2014-01-15  6219  6236  6184  6235   9167300   6235
7    2014-01-16  6263  6325  6236  6261   8070200   6261
8    2014-01-17  6260  6262  6195  6200   8076700   6200
9    2014-01-20  6219  6229  6201  6206   5785300   6206
10   2014-01-21  6211  6329  6211  6300  11308200   6300
11   2014-01-22  6300  6363  6258  6340  11254600   6340
12   2014-01-23  6350  6362  6256  6256   8837200   6256
13   2014-01-24  6168  6198  6150  6170  14605000   6170
14   2014-01-27  6047  6077  6032  6039  13821100   6039
15   2014-01-28  6035  6063  6007  6018  10986000   6018
16   2014-01-29  6094  6146  60

pandasモジュールで読み込んだデータはdataframe型式なので，numpy.array形式に変換します．これは`np.array(dataframe)`するだけです．

In [5]:
data7203_2014 = np.array(csv7203_2014)
print(data7203_2014)

[['2014-01-06' 6360 6400 ... 6300 12249300 6300]
 ['2014-01-07' 6270 6340 ... 6270 7891400 6270]
 ['2014-01-08' 6310 6320 ... 6300 7184100 6300]
 ...
 ['2014-12-26' 7629 7700 ... 7696 6069100 7696]
 ['2014-12-29' 7740 7746 ... 7662 9942800 7662]
 ['2014-12-30' 7652 7674 ... 7558 7821200 7558]]


同様のことを繰り返します．より長い期間を読み込むならosモジュール等を用いてファイルリストを取得したほうがいいと思います．

In [6]:
csv7203_2015 = pd.read_csv(
    "C:/Users/issei/Projects/BitTrader/Stock_Data/7203_2015.csv",
    encoding='SHIFT_JIS',
    header=1,
)
data7203_2015 = np.array(csv7203_2015)

csv7203_2016 = pd.read_csv(
    "C:/Users/issei/Projects/BitTrader/Stock_Data/7203_2016.csv",
    encoding='SHIFT_JIS',
    header=1
)
data7203_2016 = np.array(csv7203_2016)

csv7203_2017 = pd.read_csv(
    "C:/Users/issei/Projects/BitTrader/Stock_Data/7203_2017.csv",
    encoding='SHIFT_JIS',
    header=1
)
data7203_2017 = np.array(csv7203_2017)

csv7203_2018 = pd.read_csv(
    "C:/Users/issei/Projects/BitTrader/Stock_Data/7203_2018.csv",
    encoding='SHIFT_JIS',
    header=1
)
data7203_2018 = np.array(csv7203_2018)

In [7]:
print(data7203_2015)
print(data7203_2016)
print(data7203_2017)
print(data7203_2018)

[['2015-01-05' 7565 7575 ... 7507 9515300 7507]
 ['2015-01-06' 7322 7391 ... 7300 12387900 7300]
 ['2015-01-07' 7256 7485 ... 7407 11465400 7407]
 ...
 ['2015-12-28' 7489 7524 ... 7484 4516400 7484]
 ['2015-12-29' 7461 7498 ... 7454 9706400 7454]
 ['2015-12-30' 7497 7521 ... 7488 5710600 7488]]
[['2016-01-04' 7400 7495 ... 7337 9165700 7337]
 ['2016-01-05' 7268 7293 ... 7226 10405700 7226]
 ['2016-01-06' 7220 7259 ... 7083 10349700 7083]
 ...
 ['2016-12-28' 7000 7020 ... 6970 4747600 6970]
 ['2016-12-29' 6935 6960 ... 6838 11400700 6838]
 ['2016-12-30' 6829 6913 ... 6878 6708800 6878]]
[['2017-01-04' 7010 7103 ... 7097 9547600 7097]
 ['2017-01-05' 7090 7091 ... 7049 7644000 7049]
 ['2017-01-06' 6840 6948 ... 6930 11586100 6930]
 ...
 ['2017-12-27' 7241 7277 ... 7274 2339900 7274]
 ['2017-12-28' 7272 7277 ... 7221 2422100 7221]
 ['2017-12-29' 7231 7243 ... 7213 2618000 7213]]
[['2018-01-04' 7300 7413 ... 7413 9355400 7413]
 ['2018-01-05' 7450 7555 ... 7552 8616400 7552]
 ['2018-01-09' 7

データをまとめるには，`np.vstack`を用います．`np.vstack( (配列1, 配列2, ... , 配列n) )`というように，まとめたい配列をかっこの中に入れてください．
括弧が2重になることに注意です！
ちなみに`np.vstack`は配列の列方向(縦方向)に配列を繋ぐ関数です．なので，列数が異なる配列同士を繋げることはできません．
また，行方向（横方向）に配列を繋ぐ場合には，`np.hstack`が使えます．

In [8]:
data_7203 = np.vstack((
    data7203_2014,
    data7203_2015,
    data7203_2016,
    data7203_2017,
    data7203_2018
))
print(data_7203)

[['2014-01-06' 6360 6400 ... 6300 12249300 6300]
 ['2014-01-07' 6270 6340 ... 6270 7891400 6270]
 ['2014-01-08' 6310 6320 ... 6300 7184100 6300]
 ...
 ['2018-12-04' 7049 7069 ... 6911 7465100 6911]
 ['2018-12-05' 6844 6931 ... 6910 6383900 6910]
 ['2018-12-06' 6947 7007 ... 6876 7798100 6876]]


ところで，このままでは日時の比較ができません．なぜなら，日付が文字列になっているからです（'2014-01-06'と，コロンが付いています）．なので，これを`datetime`型に変換します．これには，`datetime.datetime.strptime`を使います．

In [9]:
from datetime import datetime as dt

`datetime.datetime`というように，2回`datetime`と書くことに注意です．
`strptime`はある型式で表記された日時の文字列を時間型に変換してくれます．

In [10]:
dt.strptime("2018-12-07", "%Y-%m-%d")

datetime.datetime(2018, 12, 7, 0, 0)

これを配列の各行の0列目に用いたいです．`for`を用いて各行を読み込むこともできますが，1行で同じことができます．
ここでは`np.frompyfunc`関数を用いましょう．

In [11]:
data_7203[:,0] = np.frompyfunc(dt.strptime, 2,1)(data_7203[:,0], "%Y-%m-%d")

In [12]:
print(data_7203)

[[datetime.datetime(2014, 1, 6, 0, 0) 6360 6400 ... 6300 12249300 6300]
 [datetime.datetime(2014, 1, 7, 0, 0) 6270 6340 ... 6270 7891400 6270]
 [datetime.datetime(2014, 1, 8, 0, 0) 6310 6320 ... 6300 7184100 6300]
 ...
 [datetime.datetime(2018, 12, 4, 0, 0) 7049 7069 ... 6911 7465100 6911]
 [datetime.datetime(2018, 12, 5, 0, 0) 6844 6931 ... 6910 6383900 6910]
 [datetime.datetime(2018, 12, 6, 0, 0) 6947 7007 ... 6876 7798100 6876]]


これで，各行の日付が日時型に変換されました．

続いて，NY市場ダウ平均株価（5年間）を読み込みます．

In [81]:
csv_dji_13_18 = pd.read_csv(
    "C:/Users/issei/Projects/BitTrader/Stock_Data/DJI_131207_181207.csv",
    encoding='UTF-8',
    header=0
)
print(csv_dji_13_18)

            Date          Open          High           Low         Close  \
0     2013-12-06  15825.549805  16022.349609  15825.549805  16020.200195   
1     2013-12-09  16019.490234  16058.400391  16015.290039  16025.530273   
2     2013-12-10  16024.120117  16029.059570  15969.530273  15973.129883   
3     2013-12-11  15970.750000  15997.219727  15827.700195  15843.530273   
4     2013-12-12  15844.820313  15845.110352  15703.790039  15739.429688   
5     2013-12-13  15745.660156  15792.799805  15717.919922  15755.360352   
6     2013-12-16  15759.599609  15930.309570  15759.599609  15884.570313   
7     2013-12-17  15884.059570  15917.959961  15836.450195  15875.259766   
8     2013-12-18  15876.570313  16173.040039  15808.919922  16167.969727   
9     2013-12-19  16162.509766  16194.719727  16121.540039  16179.080078   
10    2013-12-20  16178.570313  16287.839844  16178.570313  16221.139648   
11    2013-12-23  16225.250000  16318.110352  16225.250000  16294.610352   
12    2013-1

In [14]:
data_dji_13_18 = np.array(csv_dji_13_18)
data_dji_13_18[:,0] = np.frompyfunc(dt.strptime, 2,1)(data_dji_13_18[:,0], "%Y-%m-%d")
print(data_dji_13_18)

[[datetime.datetime(2013, 12, 6, 0, 0) 15825.549805 16022.349609 ...
  16020.200195 16020.200195 98260000]
 [datetime.datetime(2013, 12, 9, 0, 0) 16019.490234 16058.400391 ...
  16025.530272999998 16025.530272999998 91810000]
 [datetime.datetime(2013, 12, 10, 0, 0) 16024.120116999999 16029.05957
  ... 15973.129883000001 15973.129883000001 79420000]
 ...
 [datetime.datetime(2018, 12, 3, 0, 0) 25779.570313 25980.210938 ...
  25826.429688 25826.429688 388480000]
 [datetime.datetime(2018, 12, 4, 0, 0) 25752.560547 25773.119141 ...
  25027.070313 25027.070313 418900000]
 [datetime.datetime(2018, 12, 6, 0, 0) 24737.419922 24951.009766 ...
  24947.669922 24947.669922 471690000]]


In [15]:
csv_nasdaq_13_18 = pd.read_csv(
    "C:/Users/issei/Projects/BitTrader/Stock_Data/NASDAQ_131207_181207.csv",
    encoding='UTF-8',
    header=0
)
data_nasdaq_13_18 = np.array(csv_dji_13_18)
data_nasdaq_13_18[:,0] = np.frompyfunc(dt.strptime, 2,1)(data_nasdaq_13_18[:,0], "%Y-%m-%d")
print(data_nasdaq_13_18)

[[datetime.datetime(2013, 12, 6, 0, 0) 15825.549805 16022.349609 ...
  16020.200195 16020.200195 98260000]
 [datetime.datetime(2013, 12, 9, 0, 0) 16019.490234 16058.400391 ...
  16025.530272999998 16025.530272999998 91810000]
 [datetime.datetime(2013, 12, 10, 0, 0) 16024.120116999999 16029.05957
  ... 15973.129883000001 15973.129883000001 79420000]
 ...
 [datetime.datetime(2018, 12, 3, 0, 0) 25779.570313 25980.210938 ...
  25826.429688 25826.429688 388480000]
 [datetime.datetime(2018, 12, 4, 0, 0) 25752.560547 25773.119141 ...
  25027.070313 25027.070313 418900000]
 [datetime.datetime(2018, 12, 6, 0, 0) 24737.419922 24951.009766 ...
  24947.669922 24947.669922 471690000]]


## データをひとまとめにする

1. データ初日を比較する
とりあえず，各データの初日の日付を見てみましょう．

In [16]:
print(data_7203[0,0])
print(data_nasdaq_13_18[0,0])
print(data_dji_13_18[0,0])

2014-01-06 00:00:00
2013-12-06 00:00:00
2013-12-06 00:00:00


つまり，今回は`2014-01-06`を見つけたいところです．

In [17]:
t0 = max(data_7203[0,0],data_nasdaq_13_18[0,0],data_dji_13_18[0,0])
print(t0)

2014-01-06 00:00:00


In [18]:
dt_7203 = t0 - data_7203[0,0]
dt_nasdaq =  t0 - data_nasdaq_13_18[0,0]
dt_dji = t0 -data_dji_13_18[0,0]
print(dt_7203, dt_nasdaq, dt_dji)

0:00:00 31 days, 0:00:00 31 days, 0:00:00


In [19]:
int_dt_7203 = dt_7203.days
int_dt_nasdaq = dt_nasdaq.days
int_dt_dji = dt_dji.days
print(int_dt_7203, int_dt_nasdaq, int_dt_dji)

0 31 31


In [20]:
print(data_7203[int_dt_7203,0])
print(data_nasdaq_13_18[int_dt_nasdaq,0])
print(data_dji_13_18[int_dt_dji,0])

2014-01-06 00:00:00
2014-01-23 00:00:00
2014-01-23 00:00:00


日付が一致しません．依然としてずれていますね．これは，各指標が常に同じ営業日で進むとは限らないからです．

In [51]:
id0_7203 = 0
while data_7203[id0_7203,0] != t0:
    int_dt_7302 = (t0 -data_7302_13_18[int_dt_7302,0]).days
    id0_7203 -= int_dt_7203
print(id0_7203)
print(data_7203[id0_7203,0])
    
id0_nasdaq = 0
while data_nasdaq_13_18[id0_nasdaq,0] != t0:
    int_dt_nasdaq = (t0 -data_nasdaq_13_18[id0_nasdaq,0]).days
    id0_nasdaq += int_dt_nasdaq
print(id0_nasdaq)
print(data_nasdaq_13_18[id0_nasdaq,0])
    
id0_dji = 0
while data_dji_13_18[id0_dji,0] != t0:
    int_dt_dji = (t0 -data_dji_13_18[id0_dji,0]).days
    id0_dji += int_dt_dji
print(id0_dji)
print(data_dji_13_18[id0_dji,0])

0
2014-01-06 00:00:00
19
2014-01-06 00:00:00
19
2014-01-06 00:00:00


In [33]:
tf = min(data_7203[data_7203.shape[0]-1,0],data_nasdaq_13_18[data_nasdaq_13_18.shape[0]-1,0],data_dji_13_18[data_dji_13_18.shape[0]-1,0])
print(tf)

2018-12-06 00:00:00


In [80]:
idf_7203 = data_7203.shape[0]-1
while data_7203[idf_7203,0] != tf:
    int_dt_7203 = (tf -data_7203[idf_7203,0]).days
    idf_7203 -= int_dt_7203
print(idf_7203)
print(data_7203[idf_7203,0])
    
idf_nasdaq = data_nasdaq_13_18.shape[0]-1
while data_nasdaq_13_18[idf_nasdaq,0] != tf:
    int_dt_nasdaq = (tf -data_nasdaq_13_18[id0_nasdaq,0]).days
    idf_nasdaq += int_dt_nasdaq
print(idf_nasdaq)
print(data_nasdaq_13_18[idf_nasdaq,0])
    
idf_dji = data_dji_13_18.shape[0]-1
while data_dji_13_18[idf_dji,0] != tf:
    int_dt_dji = (tf -data_dji_13_18[id0_dji,0]).days
    idf_dji += int_dt_dji
print(idf_dji)
print(data_dji_13_18[idf_dji,:])

1206
2018-12-06 00:00:00
1258
2018-12-06 00:00:00
1258
[datetime.datetime(2018, 12, 6, 0, 0) 24737.419922 24951.009766
 24242.220703 24947.669922 24947.669922 471690000]


In [42]:
from datetime import timedelta as td

In [82]:

arrDatas = np.zeros(((data_dji_13_18[idf_dji,0] - data_dji_13_18[id0_dji,0]).days, 15), dtype=float)

id_arr = 0
id_7203 = id0_7203
id_dji = id0_dji
id_nasdaq = id0_nasdaq
day = t0

while  (id_7203 < idf_7203) & (id_dji < idf_dji) & (id_nasdaq < idf_nasdaq):
    if day == data_7203[id_7203,0]:
        id_7203 += 1
    if day == data_dji_13_18[id_dji,0]:
        id_dji += 1
    if day == data_nasdaq_13_18[id_nasdaq,0]:
        id_nasdaq += 1

    arrDatas[id_arr, 0:5] = data_7203[id_7203, 1:6]
    arrDatas[id_arr, 5:8] = data_dji_13_18[id_dji, 1:4]
    arrDatas[id_arr, 8]   = data_dji_13_18[id_dji, 5]
    arrDatas[id_arr, 9]   = data_dji_13_18[id_dji, 6]
    arrDatas[id_arr, 10:13] = data_nasdaq_13_18[id_nasdaq, 1:4]
    arrDatas[id_arr, 13] = data_nasdaq_13_18[id_nasdaq, 5]
    arrDatas[id_arr, 14] = data_nasdaq_13_18[id_nasdaq, 6]
    
    day += td(days=1)
    id_arr += 1
    
print(arrDatas[0,:])

    

    

[6.27000000e+03 6.34000000e+03 6.26000000e+03 6.27000000e+03
 7.89140000e+06 1.64290195e+04 1.65623203e+04 1.64290195e+04
 1.65309395e+04 8.12700000e+07 1.64290195e+04 1.65623203e+04
 1.64290195e+04 1.65309395e+04 8.12700000e+07]
